In [137]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_210706/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [138]:
import pandas as pd
pandas_seed = 42
import numpy as np
import timeit

import random
import string

import os

import torch
from torch import nn, save
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
torch.manual_seed(42)

from torchvision.transforms import ToTensor

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [139]:
result_log = []

DNA_ONE_HOT = {'A': [0, 0, 0, 1],
               'C': [0, 0, 1, 0],
               'G': [0, 1, 0, 0],
               'T': [1, 0, 0, 0]
               }



PADDING = True 
    
MAX_LEN = 60
N_FEATURES = 4
N_percentage = 0.25 # 1 - 1M

DATA_USED_N = 1000000 # max is 1000000

In [140]:
# data_org = pd.read_csv("./src/variable_length_dataset.csv", names=["strand","energy"],dtype={0: str, 1: float})
# data_org = pd.read_csv("./variable_length_dataset_1M.csv", names=["strand","energy"],dtype={0: str, 1: float})
data_org = pd.read_csv("./data/1mill_dataset.csv", names=["strand","energy"],dtype={0: str, 1: float})
print("len(data_org)=",len(data_org))
data_org.head()

len(data_org)= 1000000


,strand,energy
0,GAATCTTCGCACTCTAGCTGACCGCCTTCAGTAGTACGAATCTGGA...,-5.329171
1,AAGAAGCGGGAAGCATATCTTTATTCAGTTCCTAAT,-1.687614
2,CCTAATCGGGAATTGTTCCTCTTCCATTTGTAATGGTTATAAGAGG...,-7.224524
3,CACCGGGGCATTTATCCCGGGCTCGAAGGAAGTCTTGG,-7.222121
4,CTCGGGAAGCAAAACCCCTAG,-1.601845


In [141]:
shuffled_df = data_org.sample(frac = 1, random_state=pandas_seed)
shuffled_df.reset_index(inplace=True, drop=True)

# # Clip data, too large can not train in local

shuffled_df = shuffled_df.loc[:DATA_USED_N-1]


train_pct = 0.7
val_pct = 0.15
test_pct = 1 - train_pct - val_pct

# # shuffled_df.head()
all_n = len(shuffled_df)
split1 = int(train_pct * all_n) 
split2 = int((train_pct + val_pct) * all_n) 

train_df_org = shuffled_df.loc[0:split1-1].copy()
val_df = shuffled_df.loc[split1:split2-1].copy()
test_df = shuffled_df.loc[split2:,:].copy()

train_df_reverse = train_df_org.copy()
train_df_reverse["strand"] = train_df_reverse.loc[:,'strand'].apply(lambda x: x[::-1])

train_df_all = pd.concat([train_df_org, train_df_reverse])

print(f"total {all_n}, train idx 0 - {split1-1}, val idx{split1} - {split2-1}, test idx {split2} - {all_n-1}")
print(f"\n\nlen(train_df) : {len(train_df_org) }, \ntrain_df_all(with reverse) {len(train_df_all)} \nlen(val_df) : {len(val_df)},\nlen(test_df) : {len(test_df)}")


train_df = train_df_all

train_val_df = shuffled_df.loc[:split2-1]
train_val_mean = train_val_df["energy"].mean()
train_val_stdev = train_val_df["energy"].std()

total 1000000, train idx 0 - 699999, val idx700000 - 849999, test idx 850000 - 999999


len(train_df) : 700000, 
train_df_all(with reverse) 1400000 
len(val_df) : 150000,
len(test_df) : 150000


In [142]:
def x_transform(dna_str, pad=PADDING):
    n_pad = MAX_LEN - len(dna_str) 
    target = np.array(list(dna_str))
    onehot = np.array([DNA_ONE_HOT[letter] for letter in dna_str])
#     one_hot_paded = np.pad(onehot,((0,n_pad),(0,0)), mode='constant')
    if pad == True:
        return np.pad(onehot,((0,n_pad),(0,0)), mode='constant')
    else:
        return onehot          
    
def y_transform(energy_float):
    y_trans = (energy_float - train_val_mean) / train_val_stdev
    return y_trans   

def y_transform_reverce(y_array):
    y_arr = y_array.cpu().numpy()
    y_pred = y_arr * train_val_stdev + train_val_mean
    return y_pred

In [143]:
class TransformDataset(Dataset):
    def __init__(self, df, x_transform, y_transform):
        self.x_transform = x_transform
        self.y_transform = y_transform
        self.X = df["strand"].values
        self.y = df["energy"].values

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx): 
        x = self.x_transform(self.X[idx])
        y = self.y_transform(self.y[idx])

        return torch.tensor(x), torch.tensor(y)

# train_dataset = GetData(train_df, x_transform, y_transform)
# X,y = train_dataset[1]
# print(X.shape)
# print(y)

train_dataset = TransformDataset(train_df, x_transform, y_transform)
val_dataset = TransformDataset(val_df, x_transform, y_transform)
test_dataset = TransformDataset(test_df, x_transform, y_transform)


# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# X,y = next(iter(train_loader))
# print("Features shape:", X.shape)
# print("Target shape:", y.shape)

In [144]:
class LSTM_DNA(nn.Module):
    def __init__(self, input_dim_n, hidden_units, num_layers,is_bi_dir):
        super(LSTM_DNA, self).__init__()
        self.input_dim = input_dim_n
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.direction_n = 1 + int(is_bi_dir)
        
        self.lstm = nn.LSTM(
            input_size = input_dim_n,
            hidden_size = hidden_units,
            batch_first = True,
            num_layers = self.num_layers,
            bidirectional = is_bi_dir
        )
        
        self.linear = nn.Linear(in_features = self.hidden_units, out_features=1)
        
    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers * self.direction_n, batch_size, self.hidden_units).to(device).requires_grad_()
        c0 = torch.zeros(self.num_layers * self.direction_n, batch_size, self.hidden_units).to(device).requires_grad_()
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()
        
        return out
    
#     def validation_step(self, batch, batch_idx):
#         loss = ...
#         self.log("val_loss", loss)
    
# LR = 5e-5
# N_HIDDEN_UNITS = 16
# model  = LSTM_DNA(input_dim_n = N_FEATURES, hidden_units=N_HIDDEN_UNITS)
# model = model.float()
# loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr = LR)



In [145]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0.0
    model.train()
    
    batch_i = 1
    model = model.float()
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        output = model(X.float())
        loss = loss_function(output, y.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        batch_i += 1
#         if batch_i % 200 == 0:
#             print(f"batch {batch_i}, loss = {loss.item()}")
    
    avg_loss = total_loss / num_batches 
#     print(f"Train loss: {avg_loss}")
    return avg_loss
    
def eval_model(data_loader, model, loss_function):
    num_batches = len(data_loader)
    total_loss = 0.0
    
    model.eval()
    
    model = model.float()
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(device)
            y = y.to(device)
            output = model(X.float())
            total_loss += loss_function(output, y.float()).item()
    
    avg_loss = total_loss / num_batches
#     print(f"   Evalulation loss: {avg_loss}")
    return avg_loss
    

def predict(data_loader, model):

    output = torch.tensor([])
    output = output.to(device)
    model.eval()
    model = model.float()
    with torch.no_grad():
        for X, _ in data_loader:
            X = X.to(device)
            y_out = model(X.float())
            output = torch.cat((output, y_out.float()), 0)
    
    energy = y_transform_reverce(output)
    
    return energy

def get_avg_mse(y, y_pred):
    mse = ((y - y_pred)**2).mean()
    print(f"      current mse: {mse}")
    return mse


In [146]:
# for BATCH_SIZE in [1500]:
#     for LR in [1e-2]:
#         for N_HIDDEN_UNITS in [12]:
#             for NUM_LAYERS in [1]:   150	2500	0.01	60	1	1000000	
hyper_paras_list = [
    [150, 2500, 0.01, 60, 1, True],
    [150, 2500, 0.01, 90, 1, True]
]

result_df = pd.DataFrame(columns = ["EPOCH", "BATCH_SIZE", "LR", "N_HIDDEN_UNITS", "NUM_LAYERS","data_used_n","min_val_loss_id", "min_val_loss", "test_mse", "train_loss_list", "val_loss_list"])


for paras in hyper_paras_list:
    EPOCH,BATCH_SIZE,LR,N_HIDDEN_UNITS,NUM_LAYERS, BI_DIR = paras
    
    model  = LSTM_DNA(input_dim_n = N_FEATURES, hidden_units=N_HIDDEN_UNITS, num_layers=NUM_LAYERS, is_bi_dir=BI_DIR)
    model = model.float()
    model.to(device)
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LR)
    
#     train_dataset, val_dataset, test_dataset = train_dataset.to(device), val_dataset.to(device), test_dataset.to(device)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model_list = []
    train_loss_list = []
    val_loss_list = []

    print(f"EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, is_bi_dir = {EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, BI_DIR}")
    
    min_val_loss = 1e8
    lowest_epoch = 0
    PATH = f"EPOCH{EPOCH}BATCH_SIZE{BATCH_SIZE}LR{LR}N_HIDDEN_UNITS{N_HIDDEN_UNITS}NUM_LAYERS{N_HIDDEN_UNITS}BI_DIR{BI_DIR}.pt"
    print(PATH)
    
    patient = 0
    
    for ix_epoch in range(EPOCH):
#         print(f"Epoch {ix_epoch} ")
        start = timeit.default_timer()
        train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
        val_loss = eval_model(val_loader, model, loss_function)
        
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        
        if val_loss < min_val_loss:
                if os.path.exists(PATH):
                    os.remove(PATH)
                torch.save(model.state_dict(), PATH)
                min_val_loss = val_loss
                lowest_epoch = ix_epoch
                patient = 0
        
        else:
            patient += 1
        
        stop = timeit.default_timer()
        print(f'Epoch {ix_epoch} finished val_loss{val_loss}, Time Elapse: ', stop - start) 
        
        if patient >= 10:
            break
        
    min_val_loss_id = np.argmin(val_loss_list)
    min_val_loss = val_loss_list[min_val_loss_id]
    
    energy_predict = predict(test_loader, model)
    test_mse = get_avg_mse(test_df["energy"].values,energy_predict)
    
    print(f"Total Epoch {EPOCH}\n min_val_id {min_val_loss_id}, min_val_loss {min_val_loss}, test_mse {test_mse}")

    model_result = [EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS,BI_DIR, data_used_n,min_val_loss_id, min_val_loss, test_mse, train_loss_list, val_loss_list]

    current_df = pd.DataFrame([model_result], columns = ["EPOCH", "BATCH_SIZE", "LR", "N_HIDDEN_UNITS", "NUM_LAYERS","BI_DIR","data_used_n" ,"min_val_loss_id", "min_val_loss", "test_mse", "train_loss_list", "val_loss_list"])

    result_df = pd.concat([result_df,current_df])
    letters = string.ascii_lowercase
    suffix = ''.join(random.choice(letters) for i in range(10))
    print(suffix)
    result_df.to_csv(f"2022-04-15_8_{suffix}.csv")

EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, is_bi_dir = (150, 2500, 0.01, 60, 1, True)
EPOCH150BATCH_SIZE2500LR0.01N_HIDDEN_UNITS60NUM_LAYERS60BI_DIRTrue.pt
Epoch 0 finished val_loss0.45606881578763325, Time Elapse:  161.2352032189956
Epoch 1 finished val_loss0.4042380963762601, Time Elapse:  162.53330126299988
Epoch 2 finished val_loss0.380876524746418, Time Elapse:  161.25036523101153
Epoch 3 finished val_loss0.36629230876763663, Time Elapse:  163.04564755700994
Epoch 4 finished val_loss0.3483684544761976, Time Elapse:  162.63193735701498
Epoch 5 finished val_loss0.3327577978372574, Time Elapse:  160.90211309597362
Epoch 6 finished val_loss0.3137553632259369, Time Elapse:  160.87922641198384
Epoch 7 finished val_loss0.3096236233909925, Time Elapse:  162.1670918620075
Epoch 8 finished val_loss0.30651869575182594, Time Elapse:  164.0062433530111
Epoch 9 finished val_loss0.29945084551970164, Time Elapse:  161.62492832099088
Epoch 10 finished val_loss0.2941150610645612, Time Elaps

Epoch 102 finished val_loss0.23604136432210604, Time Elapse:  160.54408928501653
Epoch 103 finished val_loss0.23685427531599998, Time Elapse:  160.81619901797967
Epoch 104 finished val_loss0.2362846131126086, Time Elapse:  162.33300464903004
      current mse: 1.2832382370351962
Total Epoch 150
 min_val_id 94, min_val_loss 0.23549275621771812, test_mse 1.2832382370351962
EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, is_bi_dir = (150, 2500, 0.01, 90, 1, True)
EPOCH150BATCH_SIZE2500LR0.01N_HIDDEN_UNITS90NUM_LAYERS90BI_DIRTrue.pt
Epoch 0 finished val_loss0.4885522370537122, Time Elapse:  176.3077507399721
Epoch 1 finished val_loss0.43393854846556984, Time Elapse:  176.54809516400564
Epoch 2 finished val_loss0.4116021985809008, Time Elapse:  175.9118258440285
Epoch 3 finished val_loss0.39143155366182325, Time Elapse:  177.38104619900696
Epoch 4 finished val_loss0.36673202365636826, Time Elapse:  176.6452424719464
Epoch 5 finished val_loss0.34188981900612514, Time Elapse:  176.56353532

In [147]:
# # for BATCH_SIZE in [1500]:
# #     for LR in [1e-2]:
# #         for N_HIDDEN_UNITS in [12]:
# #             for NUM_LAYERS in [1]:
# hyper_paras_list = [
#     [80, 2500, 0.01, 20, 1]
# ]

# result_df = pd.DataFrame(columns = ["EPOCH", "BATCH_SIZE", "LR", "N_HIDDEN_UNITS", "NUM_LAYERS","data_used_n","min_val_loss_id", "min_val_loss", "test_mse", "train_loss_list", "val_loss_list"])


# for paras in hyper_paras_list:
#     EPOCH,BATCH_SIZE,LR,N_HIDDEN_UNITS,NUM_LAYERS = paras
    
#     model  = LSTM_DNA(input_dim_n = N_FEATURES, hidden_units=N_HIDDEN_UNITS, num_layers=NUM_LAYERS)
#     model = model.float()
#     model.to(device)
#     loss_function = nn.MSELoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr = LR)
    
# #     train_dataset, val_dataset, test_dataset = train_dataset.to(device), val_dataset.to(device), test_dataset.to(device)
    
#     train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
#     test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model_list = []
#     train_loss_list = []
#     val_loss_list = []

#     print(f"EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS = {EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS}")
    
#     min_val_loss = 1e8
#     lowest_epoch = 0
#     PATH = f"EPOCH{EPOCH}BATCH_SIZE{BATCH_SIZE}LR{LR}N_HIDDEN_UNITS{N_HIDDEN_UNITS}NUM_LAYERS{N_HIDDEN_UNITS}.pt"
#     print(PATH)
    
    
#     for ix_epoch in range(EPOCH):
#         print(f"Epoch {ix_epoch} ")
#         start = timeit.default_timer()
#         train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
#         val_loss = eval_model(val_loader, model, loss_function)
        
#         train_loss_list.append(train_loss)
#         val_loss_list.append(val_loss)
        
#         if val_loss < min_val_loss:
#                 if os.path.exists(PATH):
#                     os.remove(PATH)
#                 torch.save(model.state_dict(), PATH)
#                 min_val_loss = val_loss
#                 lowest_epoch = ix_epoch
        
#         stop = timeit.default_timer()
#         print(f'Epoch finished val_loss{val_loss}, Time Elapse: ', stop - start) 
    
#     min_val_loss_id = np.argmin(val_loss_list)
#     min_val_loss = val_loss_list[min_val_loss_id]
    
#     energy_predict = predict(test_loader, model)
#     test_mse = get_avg_mse(test_df["energy"].values,energy_predict)
    
#     print(f"Total Epoch {EPOCH}\n min_val_id {min_val_loss_id}, min_val_loss {min_val_loss}, test_mse {test_mse}")

#     model_result = [EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, data_used_n,min_val_loss_id, min_val_loss, test_mse, train_loss_list, val_loss_list]

#     current_df = pd.DataFrame([model_result], columns = ["EPOCH", "BATCH_SIZE", "LR", "N_HIDDEN_UNITS", "NUM_LAYERS","data_used_n" ,"min_val_loss_id", "min_val_loss", "test_mse", "train_loss_list", "val_loss_list"])

#     result_df = pd.concat([result_df,current_df])
#     result_df.to_csv("2022-04-15_n.csv")

In [148]:


# printing lowercase
# letters = string.ascii_lowercase
# suffix = ''.join(random.choice(letters) for i in range(10))
# suffix

In [149]:
# def x_transform(dna_str, pad=True):
#     n_pad = MAX_LEN - len(dna_str) 
#     target = np.array(list(dna_str))
#     onehot = np.array([DNA_ONE_HOT[letter] for letter in dna_str])
# #     one_hot_paded = np.pad(onehot,((0,n_pad),(0,0)), mode='constant')
    
#     if pad == True:
#         return np.pad(onehot,((0,n_pad),(0,0)), mode='constant')
#     else:
#         return onehot

# str1 = "CGATT"
# x_transform(str1)

array([[0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0,